In [1]:
from model import *
from utils import *
import tensorflow as tf
import numpy as np
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags = tf.app.flags

flags.DEFINE_string('ASC', 'qb', 'restaurant or laptop or qb')
flags.DEFINE_string('DSC', 'yelp', '{yelp, twitter} for restaurant & {amazon, twitter} for laptop')
flags.DEFINE_integer('batch_size', 128, 'number of example per batch')
flags.DEFINE_float('learning_rate', 0.001, 'learning rate')
#flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
flags.DEFINE_integer('n_iter', 25, 'training iteration')
#flags.DEFINE_integer('n_iter', 2, 'training iteration')
# We slightly modify the training procedure. Feeding all DSC data in ONE epoch can get better results.
flags.DEFINE_float('gamma', 0.1, '{0.1, 0.1, 0.9, 0.2} for {res+yelp, res+twitter, laptop+amazon, laptop+twitter')
flags.DEFINE_integer('embedding_dim', 300, 'dimension of word embedding')
flags.DEFINE_integer('position_dim', 100, 'dimension of position embedding')
#flags.DEFINE_integer('max_sentence_len', 85, 'max number of tokens per sentence')
flags.DEFINE_integer('max_sentence_len', 210, 'max number of tokens per sentence')
flags.DEFINE_integer('max_target_len', 25, 'max number of tokens per target')
flags.DEFINE_float('keep_prob1', 0.5, 'dropout keep prob1')
flags.DEFINE_float('keep_prob2', 1.0, 'dropout keep prob2')
# Parameters for capsule layers.
flags.DEFINE_integer('filter_size', 3, 'filter_size')
flags.DEFINE_integer('sc_num', 16, 'sc_num')
flags.DEFINE_integer('sc_dim', 16, 'sc_dim')
flags.DEFINE_integer('cc_num',  3, 'cc_num')
flags.DEFINE_integer('cc_dim', 24, 'cc_dim')
flags.DEFINE_integer('iter_routing', 3, 'routing iteration')
flags.DEFINE_bool("reuse_embedding", False, "reuse word embedding & id, True or False")
FLAGS = flags.FLAGS

In [3]:
def main(_):
    start_time = time.time()
    info = ''
    index = 0
    
    for name, value in FLAGS.__flags.items():
        value = value.value
        if index < 19:
            info += '{}:{}  '.format(name, value)
        if index in [5, 11]:
            info += '\n'
        index += 1
    print('\n{:-^80}'.format('Parameters'))
    print(info + '\n')
    
    print('---------')
    print(FLAGS.ASC)
    
    data_path = 'data/{}/'.format(FLAGS.ASC)
    
    if not FLAGS.reuse_embedding :
        print('Initialize Word Dictionary & Embedding')
        word_dict = data_init(data_path, FLAGS.DSC)
        w2v = init_word_embeddings(data_path, word_dict, FLAGS.DSC)
    else:
        print('Reuse Word Dictionary & Embedding')
        with open(data_path + FLAGS.DSC + '_word2id.txt', 'r', encoding='utf-8') as f:
            word_dict = eval(f.read())
        w2v = np.load(data_path + FLAGS.DSC + '_word_embedding.npy')
    
    #print(w2v)
    #print(word_dict)
    
    for i in range(15):
        model = MODEL(FLAGS, w2v, word_dict, data_path)
        model.run()
    
    end_time = time.time()
    print('Running Time: {:.0f}m {:.0f}s'.format((end_time-start_time) // 60, (end_time-start_time) % 60))

In [4]:
if __name__ == '__main__':
    tf.app.run()


-----------------------------------Parameters-----------------------------------
logtostderr:False  alsologtostderr:False  log_dir:  v:0  verbosity:0  stderrthreshold:fatal  
showprefixforinfo:True  run_with_pdb:False  pdb_post_mortem:False  run_with_profiling:False  profile_file:None  use_cprofile_for_profiling:True  
only_check_args:False  op_conversion_fallback_to_while_loop:False  test_random_seed:301  test_srcdir:  test_tmpdir:/tmp/absl_testing  test_randomize_ordering_seed:  xml_output_file:  

---------
qb
Initialize Word Dictionary & Embedding
Processing train/review.txt...
Processing train/yelp_review.txt...
Processing dev/review.txt...
Processing test/review.txt...
Processing smalltest/review.txt...
max_sentence_length 149
path data/qb/



W0410 04:25:01.475369 139926851946304 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb-2/model.py:47: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



W0410 04:25:01.490577 139926851946304 module_wrapper.py:139] From /home/rmajumder/266proj/266finalproject/transcap-qb-2/model.py:54: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0410 04:25:01.522499 139926851946304 deprecation.py:506] From /home/rmajumder/266proj/266finalproject/transcap-qb-2/model.py:66: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


...

[2, 0, 0, 2, 2, 0, 2]
[2, 2, 2, 2, 2, 2, 0]

-------------------------------------Iter0--------------------------------------
train loss=35.730660, dev loss=19.297692, dev acc=0.7543, step=105
test acc=0.7555, test precision=0.5064, test recall=0.5488, test f1=0.5242
smalltest acc=0.4286, test precision=0.2500, test recall=0.3750, test f1=0.3000
max step:0, early stop step:0
[2, 0, 0, 2, 2, 0, 2]
[2, 2, 2, 2, 2, 1, 1]


/home/rmajumder/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)



-------------------------------------Iter1--------------------------------------
train loss=33.675808, dev loss=18.800348, dev acc=0.6169, step=210
test acc=0.6010, test precision=0.5762, test recall=0.5118, test f1=0.4829
smalltest acc=0.4286, test precision=0.2000, test recall=0.2500, test f1=0.2222
max step:0, early stop step:0
[2, 0, 0, 2, 2, 0, 2]
[2, 2, 2, 2, 1, 2, 1]

-------------------------------------Iter2--------------------------------------
train loss=31.108832, dev loss=15.370182, dev acc=0.6797, step=315
test acc=0.6880, test precision=0.6625, test recall=0.6309, test f1=0.5921
smalltest acc=0.2857, test precision=0.1333, test recall=0.1667, test f1=0.1481
max step:0, early stop step:0
[2, 0, 0, 2, 2, 0, 2]
[2, 2, 2, 2, 2, 1, 1]

-------------------------------------Iter3--------------------------------------
train loss=27.110877, dev loss=14.636978, dev acc=0.5923, step=420
test acc=0.5825, test precision=0.6671, test recall=0.5684, test f1=0.4670
smalltest acc=0.4286


-------------------------------------Iter22-------------------------------------
train loss=23.011249, dev loss=14.220720, dev acc=0.8059, step=2415
test acc=0.8108, test precision=0.7183, test recall=0.7429, test f1=0.7057
smalltest acc=0.2857, test precision=0.1667, test recall=0.1667, test f1=0.1667
max step:3, early stop step:0
[2, 0, 0, 2, 2, 0, 2]
[2, 2, 2, 2, 1, 1, 0]

-------------------------------------Iter23-------------------------------------
train loss=22.897665, dev loss=14.239037, dev acc=0.8002, step=2520
test acc=0.8101, test precision=0.7235, test recall=0.7537, test f1=0.7099
smalltest acc=0.2857, test precision=0.1667, test recall=0.1667, test f1=0.1667
max step:3, early stop step:1
[2, 0, 0, 2, 2, 0, 2]
[2, 2, 2, 2, 1, 1, 0]

-------------------------------------Iter24-------------------------------------
train loss=22.824060, dev loss=14.170897, dev acc=0.8093, step=2625
test acc=0.8297, test precision=0.7306, test recall=0.7601, test f1=0.7261
smalltest acc=0.2

ValueError: Variable FeatCap_SemanCap/fully_connected/weights already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "/home/rmajumder/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 1748, in __init__
    self._traceback = tf_stack.extract_stack()
  File "/home/rmajumder/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 3426, in _create_op_internal
    op_def=op_def)
  File "/home/rmajumder/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py", line 3357, in create_op
    attrs, op_def, compute_device)
  File "/home/rmajumder/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/util/deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "/home/rmajumder/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/op_def_library.py", line 794, in _apply_op_helper
    op_def=op_def)